# Starplot Demonstration

Reference: https://starplot.dev/examples/horizon-sgr/

## Core Python Libraries

In [ ]:
from datetime import datetime, timedelta

from pytz import timezone

from starplot import DSO, HorizonPlot, MapPlot, OpticPlot, Projection, _
from starplot.styles import PlotStyle, extensions
from starplot import MapPlot, Projection, _
from starplot.callables import color_by_bv
from starplot.optics import Refractor

## Test Configuration

First, we have the position of the sensor, located at the SNC RMS facility

In [ ]:
pos_lla = [-104.844, 39.544, 1790]

### ISS (Zarya) Location

Using ***Stellarium***, we identify the ISS as having the following position.

* **Right Ascension:** 11 hours, 26 minutes, 48 seconds
* **Declination:** 11 degrees, 25 minutes, 21.6 seconds
* **Time:** July 18, 2025 at `17:41:12` in Denver, Colorado

**Right Ascension:**

For Right-Ascension, we must convert the HMS notation into Decimal Degree notation.  This is a 2-step process. 

$$
h_\textit{dec} = h + \frac{m}{60} + \frac{s}{360}
$$

$$
\textit{deg} = \frac{h_\textit{dec}}{24} \cdot 360
$$

In [ ]:
ra_hms = [ 11, 26, 48 ]
ra_h_dec = ra_hms[0] + ra_hms[1]/60. + ra_hms[2]/360.
print(f'Right Ascension: {ra_h_dec} Hours')
ra_deg = (ra_h_dec / 24.) * 360.
print( f'Right Ascension: {ra_deg} degs' )

In [ ]:
dec_dms = ( 11, 25, 21.6 )
de_deg  = dec_dms[0] + dec_dms[1] / 60. + dec_dms[2] / 360.
print(f'Declination Degrees: {de_deg}' )

In [ ]:
dt = timezone("America/Denver").localize( datetime( year   = 2025,
                                         month  = 7,
                                         day    = 18,
                                         hour   = 17,
                                         minute = 41,
                                         second = 12 ) )
dt

For reference, we expect the following relative position, per Stellarium:
![Stellarium Plot](./images/ISS_Stellarium.png)

## Horizon Plot

This creates a plot using an Az/El coordinate system. Link to reference in [Starplot API is here](https://starplot.dev/examples/horizon-sgr/).  Importantly, this plot does not extend above 90-degrees of elevation.

Our plot should be a purple/pink oval near the center of the plot.

In [ ]:
style = PlotStyle().extend(
    extensions.BLUE_MEDIUM,
    extensions.MAP,
    {"figure_background_color": "hsl(212, 27%, 48%)"},
)
hplot = HorizonPlot(
    altitude=(20, 90),
    azimuth=(155, 245),
    lat= pos_lla[1],
    lon = pos_lla[0],
    dt=dt,
    style=style,
    resolution=3000,
    scale=0.9,
)

hplot.constellations()
hplot.milky_way()

hplot.stars(where=[_.magnitude < 5], where_labels=[_.magnitude < 2])
#hplot.messier(where=[_.magnitude < 11], true_size=False, label_fn=lambda d: f"M{d.m}")

hplot.constellation_labels()
hplot.horizon(labels={180: "SOUTH"})
hplot.gridlines()

hplot.circle( center = ( ra_deg, de_deg ),
              radius_degrees = 1,
              style={
                  'alpha': 0.8,
                  'color': '#ed7eed',
                  'fill_color': '#ed7eed' },
    label="ISS (Zarya)",
)



hplot.export("images/horizon_iss.png", padding=0.2)

##  Orthographic Map Plot

This isn't a particularly useful plot.

In [ ]:
mplot = MapPlot(
    projection=Projection.ORTHOGRAPHIC,
    lat=pos_lla[1],
    lon=pos_lla[0],
    dt=dt,
    ra_min  = ra_deg - 25,
    ra_max  = ra_deg + 25,
    dec_min = de_deg - 25,
    dec_max = de_deg + 25,
    style=style,
    scale=0.9,  # lower the scale since it shows a large area
)
mplot.gridlines(labels=True)
mplot.constellations()
mplot.constellation_borders()

mplot.stars(where=[_.magnitude < 8], where_labels=[_.magnitude < 5])
mplot.open_clusters(where=[_.magnitude < 12], true_size=False, labels=None)
mplot.galaxies(where=[_.magnitude < 12], true_size=False, labels=None)
mplot.nebula(where=[_.magnitude < 12], true_size=False, labels=None)

mplot.constellation_labels(style__font_alpha=0.4)
mplot.ecliptic()
mplot.celestial_equator()
mplot.milky_way()
mplot.planets()

mplot.marker(
    ra  = ra_deg,
    dec = de_deg,
    style={
        "marker": {
            "size": 80,
            "symbol": "circle",
            "fill": "full",
            "color": "#ed7eed",
            "edge_color": "#e0c1e0",
            "alpha": 0.8,
        },
        "label": {
            "font_size": 25,
            "font_weight": "bold",
            "font_color": "#c83cc8",
            "font_alpha": 1,
        },
    },
    label="ISS (Zarya)",
)

mplot.export("map_orthographic.png", padding=0.3, transparent=True)

## Zenith Starmap


In [ ]:
from datetime import datetime
from pytz import timezone
from starplot import MapPlot, Projection, Star, DSO, _
from starplot.styles import PlotStyle, extensions

p = MapPlot(
    projection=Projection.ZENITH,
    lat = pos_lla[1],
    lon = pos_lla[0],
    dt  = dt,
    style=PlotStyle().extend(
        extensions.BLUE_GOLD,
    ),
    resolution=3000,
    autoscale=True,
)
p.horizon()
p.constellations()
p.stars(where=[_.magnitude < 4.6], where_labels=[_.magnitude < 2.1])

p.galaxies(where=[_.magnitude < 9], true_size=False, labels=None)
p.open_clusters(where=[_.magnitude < 9], true_size=False, labels=None)

p.constellation_borders()
p.ecliptic()
p.celestial_equator()
p.milky_way()

p.marker(
    ra= ra_deg,
    dec= de_deg,
    style={
        "marker": {
            "size": 120,
            "symbol": "circle",
            "fill": "none",
            "color": None,
            "edge_color": "hsl(44, 70%, 73%)",
            "edge_width": 6,
            "line_style": [1, [2, 3]],
            "alpha": 1,
            "zorder": 100,
        },
        "label": {
            "zorder": 200,
            "font_size": 22,
            "font_weight": "bold",
            "font_color": "hsl(44, 70%, 64%)",
            "font_alpha": 1,
            "offset_x": "auto",
            "offset_y": "auto",
            "anchor_point": "top right",
        },
    },
    label="ISS (Zarya)",
)
p.constellation_labels()

p.export("star_chart_detail.png", transparent=True, padding=0.1)

## Optical Plot

This creates a more helpful view relative

In [ ]:
style = PlotStyle().extend(
    extensions.GRAYSCALE_DARK,
    extensions.OPTIC,
)

oplot = OpticPlot(
    ra  = ra_deg,
    dec = de_deg,
    lat = pos_lla[1],
    lon = pos_lla[0],
    
    optic=Refractor(
        focal_length=10,
        eyepiece_focal_length=1,
        eyepiece_fov=45,
    ),
    dt=dt,
    style=style,
    resolution=3096,
    autoscale=True,
)

oplot.circle( center = ( ra_deg, de_deg ),
              radius_degrees = 0.05,
              style={
                  'alpha': 0.8,
                  'color': '#ed7eed',
                  'fill_color': '#ed7eed' },
    label="ISS (Zarya)",
)

oplot.stars(where=[_.magnitude < 14], color_fn=color_by_bv, bayer_labels=True)
oplot.dsos(where=[_.magnitude < 4.1], labels=None)
oplot.info()
oplot.export("images/optical_plot_iss.png", padding=0.1, transparent=True)